In [5]:
import openai

In [6]:
import os
import sys

In [7]:
from langchain.document_loaders import PyPDFLoader

In [8]:
loader = PyPDFLoader("docs/SOP-CUHK.pdf")
pages = loader.load()

In [9]:
len(pages)

2

In [10]:
page = pages[0]
print(page.page_content[0:200])

Statement  of  Purpose  
 During  my  undergraduate  summer  vacations,  I  had  completed  a  deep  learning  course  and  was  
busy
 
experimenting
 
with
 
some
 
algorithms
 
when
 
I
 
heard
 
m


In [11]:
page.metadata

{'producer': 'Skia/PDF m133 Google Docs Renderer',
 'creator': 'PyPDF',
 'creationdate': '',
 'title': 'SOP-CUHK',
 'source': 'docs/SOP-CUHK.pdf',
 'total_pages': 2,
 'page': 0,
 'page_label': '1'}

In [12]:
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.blob_loaders import FileSystemBlobLoader
from langchain_community.document_loaders.parsers.audio import OpenAIWhisperParserLocal
from langchain_community.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

In [13]:
url="https://www.youtube.com/watch?v=3LopI4YeC4I"
save_dir="docs/youtube/"
loader = GenericLoader(
    # YoutubeAudioLoader([url], save_dir),  # Fetch audio from YouTube
    FileSystemBlobLoader(save_dir, glob="*.m4a"),
    OpenAIWhisperParserLocal(device="cpu")  # Use 'cpu' or 'cuda' based on your setup
)
docs = loader.load()

Using the following model:  openai/whisper-base


Device set to use cpu


Transcribing part docs/youtube/Is Success Luck or Hard Work？.m4a!


/opt/homebrew/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [14]:
docs

[Document(metadata={'source': 'docs/youtube/Is Success Luck or Hard Work？.m4a'}, page_content=" During the COVID lockdown, this headline went viral. Nearly half of men say they do most of the homeschooling. 3% of women agree. I bring this up, not to debate who's right, but because it's a great example of something called egocentric bias. Most people think they do most of the work. For example, researchers have asked authors of multi-author papers what percentage of the work they personally did. And when they add up those percentages, the sum is on average 140%. When couples are asked to estimate how much of the housework they do, the combined total is almost always over 100%. Now you might think this is because people want to appear more helpful than they actually are. But that's not it. When couples are asked what fraction of the fights they start or how much of the mess is theirs, the total is again over a hundred. People think they do more of the work, but they also think they cause

In [15]:
len(docs)

1

In [16]:
docs[0].page_content[0:500]

" During the COVID lockdown, this headline went viral. Nearly half of men say they do most of the homeschooling. 3% of women agree. I bring this up, not to debate who's right, but because it's a great example of something called egocentric bias. Most people think they do most of the work. For example, researchers have asked authors of multi-author papers what percentage of the work they personally did. And when they add up those percentages, the sum is on average 140%. When couples are asked to e"

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [18]:
chunk_size =450
chunk_overlap = 0

In [19]:
# r_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=chunk_size,
#     chunk_overlap=chunk_overlap,
#     separators=["\n\n", "\n", "(?<=\. )", " ", ""]  
# )

SyntaxError: invalid non-printable character U+00A0 (2211860444.py, line 4)

In [27]:
# r_splitter.split_text(docs[0].page_content[0:500])

["During the COVID lockdown, this headline went viral. Nearly half of men say they do most of the homeschooling. 3% of women agree. I bring this up, not to debate who's right, but because it's a great example of something called egocentric bias. Most people think they do most of the work. For example, researchers have asked authors of multi-author papers what percentage of the work they personally did. And when they add up those percentages, the",
 'sum is on average 140%. When couples are asked to e']

In [20]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import JinaEmbeddings

In [21]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

class JinaEmbeddings:
    def __init__(self, model_name="jinaai/jina-embeddings-v2-base-en"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        self.model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

    def embed_documents(self, texts):
        inputs = self.tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model(**inputs)
            embeddings = F.normalize(outputs.last_hidden_state.mean(dim=1), p=2, dim=1)
        return embeddings.cpu().numpy().tolist()

    def embed_query(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
            embedding = F.normalize(outputs.last_hidden_state.mean(dim=1), p=2, dim=1)
        return embedding.cpu().numpy()[0].tolist()

In [22]:
from langchain.vectorstores import Chroma

persist_directory = 'docs/chroma/'

embedding = JinaEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

/var/folders/bp/s_kwl8x13sb449hbbjskcnwc0000gn/T/ipykernel_7321/3486253483.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)


In [23]:
docs

[Document(metadata={'source': 'docs/youtube/Is Success Luck or Hard Work？.m4a'}, page_content=" During the COVID lockdown, this headline went viral. Nearly half of men say they do most of the homeschooling. 3% of women agree. I bring this up, not to debate who's right, but because it's a great example of something called egocentric bias. Most people think they do most of the work. For example, researchers have asked authors of multi-author papers what percentage of the work they personally did. And when they add up those percentages, the sum is on average 140%. When couples are asked to estimate how much of the housework they do, the combined total is almost always over 100%. Now you might think this is because people want to appear more helpful than they actually are. But that's not it. When couples are asked what fraction of the fights they start or how much of the mess is theirs, the total is again over a hundred. People think they do more of the work, but they also think they cause

In [24]:
vectordb.add_documents(docs)

['356ed5cd-4410-4e0c-ac6d-86eb9415e6c8']

In [25]:
print(vectordb._collection.count())

3


In [26]:
question = "What happened in lockdown?"
t = vectordb.similarity_search(question,k=3)
len(t)

3

In [27]:
t[0].page_content

" During the COVID lockdown, this headline went viral. Nearly half of men say they do most of the homeschooling. 3% of women agree. I bring this up, not to debate who's right, but because it's a great example of something called egocentric bias. Most people think they do most of the work. For example, researchers have asked authors of multi-author papers what percentage of the work they personally did. And when they add up those percentages, the sum is on average 140%. When couples are asked to estimate how much of the housework they do, the combined total is almost always over 100%. Now you might think this is because people want to appear more helpful than they actually are. But that's not it. When couples are asked what fraction of the fights they start or how much of the mess is theirs, the total is again over a hundred. People think they do more of the work, but they also think they cause more of the problems. So why is this? I think it's simply because you experience and remember

In [28]:
from langchain.chains import RetrievalQA

In [29]:
api_key = ''

In [35]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.0, google_api_key=api_key)

In [36]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [37]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [38]:
question = "What happened in lockdown?"


In [39]:
result = qa_chain({"query": question})

In [43]:
result['result']

'During the COVID lockdown, a headline went viral stating that nearly half of men said they did most of the homeschooling, while only 3% of women agreed. This discrepancy was used as an example of egocentric bias. Thanks for asking!'

In [44]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

/var/folders/bp/s_kwl8x13sb449hbbjskcnwc0000gn/T/ipykernel_7321/1956280058.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [45]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [46]:
question = "What is happening in this video?"
result = qa({"question": question})

In [47]:
result['answer']


"The video discusses egocentric bias, which is the tendency for people to overestimate their own contributions and underestimate others. It explores how this bias affects our perception of luck and success, using examples like homeschooling during COVID, hockey players' birthdates, and the selection of NASA astronauts. The video also touches on how downplaying the role of chance can paradoxically improve one's probability of success and how successful people may be less inclined to give back due to a distorted view of reality. Finally, it suggests that acknowledging fortunate circumstances makes us more likable and happier."

In [48]:
question = "Is success luck or hard work?"
result = qa({"question": question})

In [49]:
result['answer']

'The truth is you need both skill/hard work and luck to be successful.'

In [69]:
vectordb.add_documents(pages)

['3ec12c82-140b-4798-b103-24387e34d1e0',
 '8d51180e-088e-4516-ba1f-eaee5446a75f']

In [71]:
question = "What is the SOP about?"
result = qa({"question": question})
result['answer']

"The statement of purpose is about the applicant's journey and passion for data science, particularly its potential to transform healthcare from a reactive to a proactive approach. It highlights their experience in using AI for early diagnosis, their work in healthcare consulting, and their desire to further explore the integration of AI systems with healthcare for clinical diagnosis through CUHK's Master's program in Advanced Studies in Statistics and Data Science."